## Change the default torch version for compatibility

In [ ]:
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

## Clone Detectron2 repo

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

## Resize the sample image to required square shape

In [2]:
import cv2
import matplotlib.pyplot as plt
I1= cv2.imread('sample_image.jpg')

# change 960 to what the model has been trained on
I2 = cv2.resize(I1, (960, 960))

print(I2.shape)
cv2.imwrite('new.jpg', I2)

In [3]:
# plt.imshow(I2)

## Export model to Caffe2 format

Make sure to edit **export_model.py**, change these sizes to what model has been trained on, as above for sample_image.

<code>aug = T.ResizeShortestEdge(
    [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
)</code>

to

<code>aug = T.ResizeShortestEdge(
    [960, 960], 960
)</code>

In [4]:
#files required:
# model config.yaml
# model weights .pth file
# sample resized image

!python detectron2/tools/deploy/export_model.py  --config-file config.yaml --output ./ --format onnx --sample-image new.jpg --export-method caffe2_tracing MODEL.DEVICE cuda MODEL.WEIGHTS model.pth

## Convert Caffe2 to Onnx format

Make sure to edit **create_onnx.py** file

In **get_anchors** function:
1. change *image preprocessing* to your model's image preprocessing
2. do the same for *model preprocessing* if any
3. change size (960 in this case) on line: <code>imagelist_images = ImageList.from_tensors(images, 1344)</code>


In [5]:
#files required:
# model config.yaml
# model weights .pth file
# sample resized image
# caffe2 format model from previous step (model.onnx)

!python create_onnx.py --onnx converted.onnx --exported_onnx model.onnx --det2_config config.yaml -s new.jpg --det2_weights model.pth 

## Create TensorRT engine from converted Onnx model

In [6]:
#files required:
# onnx file from previous step (converted.onnx)

!trtexec --onnx=converted.onnx --saveEngine=engine.trt --useCudaGraph

## Optional: Run TensorRT engine to analyze performance

In [7]:
!trtexec --loadEngine=engine.trt --useCudaGraph --noDataTransfers --iterations=100 --avgRuns=100

## Optional: Run inference on sample image

In [8]:
# need to change labels in this file, but not necessary if only checking the masks

!python infer.py --engine engine.trt --input new.jpg --det2_config config.yml --output output/